In [1]:
import os
import matplotlib.pyplot as plt
import math
import cv2
import numpy as np

In [129]:
data_path = '/var/tmp/zapp_an-a/Desktop/gamma_sources_identification/Full dataset/gray_stretch_dataset'
# ASSUMING WE ARE DOING BINARY CLASSIFICATION
def load_data():
    # let's put directly images inside these arrays and not only image paths
    x_train = []
    y_train = []
    x_test = []
    y_test = []

    train_bkg_count = 0
    train_src_count = 0

    for img_name in os.listdir(data_path):
        img_full_path = data_path + '/' + img_name
        image = cv2.imread(img_full_path)
        image = cv2.pyrDown(image)
        image = cv2.pyrDown(image)
        if img_name[0] == 'B': #background image
            if train_bkg_count < 800: # append in training set
                x_train.append(image)
                y_train.append(0)
                train_bkg_count += 1
            else: # append in test set
                x_test.append(image)
                y_test.append(0)
        else: #source image
            if train_src_count < 800:
                x_train.append(image)
                y_train.append(1)
                train_src_count += 1
            else: # append in test set
                x_test.append(image)
                y_test.append(1)

    # at the end of the for loop we should have the split dataset
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_test = np.array(x_test)
    y_test = np.array(y_test)
    return (x_train, y_train), (x_test, y_test)

In [130]:
(x_train, y_train), (x_test, y_test) = load_data()

In [131]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(1600, 150, 150, 3)
(1600,)
(400, 150, 150, 3)
(400,)


In [ ]:
# ok so we have the whole loaded dataset. Each image is of course made of 3 channels. Let's see how a
# possible CNN behave

In [132]:
# Normalization
x_train = x_train / 255
x_test = x_test / 255

In [18]:
# BUILING THE CONVOLUTIONAL NEURAL NETWORKS

In [10]:
import tensorflow as tf
from tensorflow import keras

2023-08-24 09:34:48.964520: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-24 09:34:48.981827: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-24 09:34:49.110028: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-24 09:34:49.110912: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-24 09:34:49.629086: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [133]:
num_of_classes = 2

model = keras.Sequential()
#before we included all the layers in the "constructor of Sequential", but we can also do model.add
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3))) # the input shape only in the first layer
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.5)) #just to fight overfitting, we drop some (neurons?)

model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Dense(num_of_classes, activation='sigmoid')) #sigmoid because we're doing binary classification.

In [134]:
# compile

model.compile(optimizer = 'adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

In [137]:
# training the neural network
history = model.fit(x_train, y_train, validation_split=0.1, epochs=10)
#few epochs for avoiding overtraining

Epoch 1/10
45/45 [==============================] - 6s 137ms/step - loss: 0.1472 - acc: 0.9361 - val_loss: 0.4684 - val_acc: 0.7500
Epoch 2/10
45/45 [==============================] - 6s 141ms/step - loss: 0.1460 - acc: 0.9306 - val_loss: 0.8949 - val_acc: 0.6500
Epoch 3/10
45/45 [==============================] - 6s 142ms/step - loss: 0.1370 - acc: 0.9375 - val_loss: 0.7643 - val_acc: 0.6812
Epoch 4/10
45/45 [==============================] - 6s 135ms/step - loss: 0.1241 - acc: 0.9451 - val_loss: 0.6896 - val_acc: 0.7250
Epoch 5/10
45/45 [==============================] - 6s 137ms/step - loss: 0.1153 - acc: 0.9486 - val_loss: 0.5274 - val_acc: 0.8313
Epoch 6/10
45/45 [==============================] - 6s 143ms/step - loss: 0.1300 - acc: 0.9444 - val_loss: 0.4900 - val_acc: 0.7750
Epoch 7/10
45/45 [==============================] - 6s 142ms/step - loss: 0.1240 - acc: 0.9417 - val_loss: 0.7785 - val_acc: 0.7125
Epoch 8/10
45/45 [==============================] - 6s 137ms/step - loss: 0.

In [138]:
loss, accuracy = model.evaluate(x_test, y_test)
print('Test Accuracy =', accuracy)

13/13 [==============================] - 1s 39ms/step - loss: 0.7437 - acc: 0.8550
Test Accuracy = 0.8550000190734863
